In [64]:
import pandas as pd
import numpy as np
import time
start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.0 seconds ---


In [146]:
#start_time = time.time()
#Phase1 Using the offline workload A as the traning data

#Step1 Load data

workload_A_all=pd.read_csv('../../offline_workload.csv') 
#print(workload_A.head())
tmp_flag = (workload_A_all != 0).any(axis=0)
workload_A = workload_A_all.loc[:, tmp_flag]
#workload_A
workload_A = workload_A.loc[:, ~(workload_A == workload_A.iloc[0]).all()]

workload_A = workload_A.loc[:, workload_A.isin([0]).mean() < .6]

knobs_list = ['k1','k2','k3','k4','k5','k6','k7','k8','s1','s2','s3','s4']


#print(workload_A_all)
#print("--- %s seconds ---" % (time.time() - start_time))

In [66]:
# Normalization
from sklearn.preprocessing import MinMaxScaler

tmpframe = workload_A.T.copy()

tmpframe1 = tmpframe.tail(314-1).T
tmpframe2 = tmpframe.head(1).T

knobs_metrics_name = tmpframe1.columns
work_id_name = tmpframe2.columns

tmp_workload_knobs_metrics = (workload_A[knobs_metrics_name]).values

dataScalar = MinMaxScaler().fit(tmp_workload_knobs_metrics)

workload_knobs_metrics_Norm = dataScalar.transform(tmp_workload_knobs_metrics)
tmp_workload_id = (workload_A[work_id_name]).values

workload_knobs_metrics_Norm_id = np.hstack((tmp_workload_id,workload_knobs_metrics_Norm))

col_list = list(work_id_name)+list(knobs_metrics_name)
norm_workload_A = pd.DataFrame(data=workload_knobs_metrics_Norm_id,columns=col_list)

latencyArray = (workload_A[['latency']]).values

latencyScalar = MinMaxScaler().fit(latencyArray)


In [67]:
start_time = time.time()

#Step2 Split workload_A by creating different dataframes

workload_A_ids = norm_workload_A["workload id"].unique()
workload_A_list = []
for num,workload_A_id in enumerate(workload_A_ids,start=1):
    tmp_workload = norm_workload_A[norm_workload_A['workload id'].isin([workload_A_id])]
    workload_A_list.append(tmp_workload)
    exec("workload_A_%s = tmp_workload"%num)

#workload_A_1
print("--- %s seconds ---" % (time.time() - start_time))


--- 0.09375143051147461 seconds ---


In [68]:
start_time = time.time()

#Step3 Get metrics

tmpframe = norm_workload_A.T.copy()
tmpframe = tmpframe.tail(314-1-12).T
metrics_name = tmpframe.columns
#print(metrics_name)
workload_A_listformetrics = []
num_workloads_A = len(workload_A_list)
for i in range(num_workloads_A):
    tmp = (workload_A_list[i])[metrics_name]
    workload_A_listformetrics.append(tmp)

print("--- %s seconds ---" % (time.time() - start_time))


--- 1.4838697910308838 seconds ---


In [153]:
start_time = time.time()

#Step1 load workload B data set

workload_B=pd.read_csv('../../online_workload_B.csv') 

#knobs_metrics_name = tmpframe1.columns
#work_id_name = tmpframe2.columns

tmp_workload_B_knobs_metrics = (workload_B[knobs_metrics_name]).values

#dataScalar = MinMaxScaler().fit(tmp_workload_knobs_metrics)

Bworkload_knobs_metrics_Norm = dataScalar.transform(tmp_workload_B_knobs_metrics)
Btmp_workload_id = (workload_B[work_id_name]).values

Bworkload_knobs_metrics_Norm_id = np.hstack((Btmp_workload_id,Bworkload_knobs_metrics_Norm))

#col_list = list(work_id_name)+list(knobs_metrics_name)
norm_workload_B = pd.DataFrame(data=Bworkload_knobs_metrics_Norm_id,columns=col_list)



print("--- %s seconds ---" % (time.time() - start_time))
norm_workload_B

--- 0.1436159610748291 seconds ---


,workload id,k1,k2,k3,k4,k5,k6,k7,k8,s1,...,worker_2.Paging_and_Virtual_Memory.pgactivate,worker_2.Paging_and_Virtual_Memory.pgalloc_normal,worker_2.Paging_and_Virtual_Memory.pgfault,worker_2.Paging_and_Virtual_Memory.pgfree,worker_2.Paging_and_Virtual_Memory.pgpgin,worker_2.Paging_and_Virtual_Memory.pgpgout,worker_2.Processes.Blocked,worker_2.Processes.Runnable,worker_2.Processes.fork,worker_2.Processes.pswitch
0,1-1,0.423077,0.176471,1,1,0.0769231,0.919048,1,0.4,0.010101,...,0.187263,0.434244,0.0489238,0.284526,1.30494,0.162126,1.18333,0.223063,0.377736,0.225861
1,1-1,0.192308,0.176471,0.5,0.5,0.0769231,0.919048,1,0.4,0.0909091,...,0.157029,0.369936,0.0443326,0.22023,1.0009,0.172732,0.857143,0.19635,0.317073,0.182314
2,1-1,0.134615,0.0294118,1,1,0.0769231,0.919048,1,0.4,0.0909091,...,0.0767531,0.245833,0.0152169,0.147634,0.861225,0.0733324,0.339815,0.0693224,0.146857,0.13885
3,1-1,0.423077,0.176471,1,1,0.0769231,0.919048,1,0.4,0.0909091,...,0.167855,0.339363,0.0470671,0.228134,0.958487,0.180253,1.68056,0.256706,0.340859,0.198389
4,1-1,0.192308,0.176471,0.5,0.5,0.0769231,0.919048,1,0.4,0.0909091,...,0.158224,0.109943,0.0385281,0.0939092,0.0074373,0.202941,0.0873016,0.201799,0.323645,0.148199
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,14-1,0.653846,1,0,0,0.0769231,0.919048,1,0.4,0.0909091,...,0.00161311,0.0686614,0.203777,0.0459204,7.07352e-06,0.0267308,0,0.287024,0.759105,0.0854303
596,14-1,0.192308,0.294118,0,0,0.0769231,0.919048,1,0.4,0.0909091,...,0.00112479,0.021567,0.150025,0.159283,0,0.0119027,0.0208333,0.210566,0.537258,0.0477424
597,14-1,0.269231,0.411765,0,0,0.0769231,0.919048,1,0.4,0.0909091,...,0.00252511,0.0370058,0.141262,0.314006,1.83154e-05,0.0180185,0.0257937,0.259081,0.63011,0.0625165
598,14-1,0.653846,1,0,0,0.0769231,0.919048,0,0.4,0.0909091,...,0.00267613,0.0738297,0.196381,0.514412,1.4147e-05,0.0316406,0.0472222,0.3592,0.795794,0.0872622


In [154]:
start_time = time.time()

#Step2 Split workload_B by creating different dataframes

workload_B_ids = norm_workload_B["workload id"].unique()
workload_B_list = []
for num,workload_B_id in enumerate(workload_B_ids,start=1):
    tmp_workload = norm_workload_B[norm_workload_B['workload id'].isin([workload_B_id])]
    workload_B_list.append(tmp_workload)
    exec("workload_B_%s = tmp_workload"%num)

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.07878994941711426 seconds ---


In [155]:
start_time = time.time()

#Step3 Split each workload into 5 data points and 1 data point as target

workload_B_mapping_data_list = []
workload_B_target_list = []

num_workloads_B = len(workload_B_list)

workload_B_mapping_data = []
workload_B_target = []

workload_B_mapping_data2 = []
workload_B_target2 = []

workload_B_mapping_data3 = []
workload_B_target3 = []

workload_B_mapping_data4 = []
workload_B_target4 = []

workload_B_mapping_data5 = []
workload_B_target5 = []

workload_B_mapping_data6 = []
workload_B_target6 = []

for i in range(num_workloads_B):
    workload_B_mapping_data.append(workload_B_list[i][:-1])
    workload_B_target.append(workload_B_list[i][5:6])

workload_B_mapping_data_list.append(workload_B_mapping_data)   
workload_B_target_list.append(workload_B_target)

for i in range(num_workloads_B):
    workload_B_mapping_data2.append(pd.concat([(workload_B_list[i][0:4]),(workload_B_list[i][5:6])]))
    workload_B_target2.append((workload_B_list[i][4:5]))

workload_B_mapping_data_list.append(workload_B_mapping_data2)   
workload_B_target_list.append(workload_B_target2)    
    
for i in range(num_workloads_B):
    workload_B_mapping_data3.append(pd.concat([(workload_B_list[i][0:3]),(workload_B_list[i][4:6])]))
    workload_B_target3.append((workload_B_list[i][3:4]))
    
workload_B_mapping_data_list.append(workload_B_mapping_data3)   
workload_B_target_list.append(workload_B_target3)

for i in range(num_workloads_B):
    workload_B_mapping_data4.append(pd.concat([(workload_B_list[i][0:2]),(workload_B_list[i][3:6])]))
    workload_B_target4.append((workload_B_list[i][2:3]))

workload_B_mapping_data_list.append(workload_B_mapping_data4)   
workload_B_target_list.append(workload_B_target4)

for i in range(num_workloads_B):
    workload_B_mapping_data5.append(pd.concat([(workload_B_list[i][0:1]),(workload_B_list[i][2:6])]))
    workload_B_target5.append((workload_B_list[i][1:2]))

workload_B_mapping_data_list.append(workload_B_mapping_data5)   
workload_B_target_list.append(workload_B_target5)

for i in range(num_workloads_B):
    workload_B_mapping_data6.append(pd.concat([(workload_B_list[i][0:0]),(workload_B_list[i][1:6])]))
    workload_B_target6.append((workload_B_list[i][0:1]))

workload_B_mapping_data_list.append(workload_B_mapping_data6)   
workload_B_target_list.append(workload_B_target6)
    
    

print("--- %s seconds ---" % (time.time() - start_time))


--- 0.41788315773010254 seconds ---


In [72]:
start_time = time.time()

#Step1 load workload C data set

workload_C=pd.read_csv('../../online_workload_C.csv') 

#knobs_metrics_name = tmpframe1.columns
#work_id_name = tmpframe2.columns

tmp_workload_C_knobs_metrics = (workload_C[knobs_metrics_name]).values

#dataScalar = MinMaxScaler().fit(tmp_workload_knobs_metrics)

Cworkload_knobs_metrics_Norm = dataScalar.transform(tmp_workload_C_knobs_metrics)
Ctmp_workload_id = (workload_C[work_id_name]).values

Cworkload_knobs_metrics_Norm_id = np.hstack((Ctmp_workload_id,Cworkload_knobs_metrics_Norm))

#col_list = list(work_id_name)+list(knobs_metrics_name)
norm_workload_C = pd.DataFrame(data=Cworkload_knobs_metrics_Norm_id,columns=col_list)



print("--- %s seconds ---" % (time.time() - start_time))
norm_workload_C

--- 0.10180282592773438 seconds ---


,workload id,k1,k2,k3,k4,k5,k6,k7,k8,s1,...,worker_2.Paging_and_Virtual_Memory.pgactivate,worker_2.Paging_and_Virtual_Memory.pgalloc_normal,worker_2.Paging_and_Virtual_Memory.pgfault,worker_2.Paging_and_Virtual_Memory.pgfree,worker_2.Paging_and_Virtual_Memory.pgpgin,worker_2.Paging_and_Virtual_Memory.pgpgout,worker_2.Processes.Blocked,worker_2.Processes.Runnable,worker_2.Processes.fork,worker_2.Processes.pswitch
0,15-6,0.192308,0.0588235,1,1,0.0769231,0.919048,1,0.4,0.0909091,...,0.00388873,0.0114641,0.0134484,0.0065893,0.00297309,0.0122871,0,0.128833,0.341439,0.0249509
1,15-6,1,1,0,0,0.0769231,0.919048,1,0.4,0.191919,...,0.00666343,0.312275,0.0770282,1.04607,0.0034006,0.0287578,0.0277778,0.485166,0.545355,0.099231
2,15-6,0.307692,0.294118,0,0,1,0.919048,1,0.4,0.0909091,...,0.00447855,0.077395,0.261988,0.184435,4.42095e-06,0.0142952,0,0.239696,0.357922,0.0547154
3,15-6,0.307692,0.294118,0,0,0.0769231,0.919048,1,0.4,0.0909091,...,0.00397231,0.0745863,0.272066,0.0406116,0,0.0145802,0,0.235295,0.376448,0.0579044
4,15-6,1,1,0,0,0.0769231,0.919048,1,0.4,0.0909091,...,0.00577951,0.158567,0.760305,0.131559,0.00869521,0.0286865,0.0296296,0.457512,0.511104,0.0850213
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,30-5,0.269231,0.176471,1,1,0.0769231,0.919048,1,0.2,0.0909091,...,0.0924577,0.216595,0.0348831,0.178325,0.00903047,0.198777,0.0645541,0.413789,0.0465767,0.0545866
496,30-5,0.0384615,0,1,1,0.0769231,0.919048,1,0.4,0.0909091,...,0.0532138,0.0705106,0.00907851,0.0584301,0,0.0728725,0.0196778,0.105379,0.00591019,0.0149903
497,30-5,0.269231,0.176471,1,1,0.0769231,0.919048,1,0.4,0.0909091,...,0.0901591,0.141134,0.0329677,0.121237,0,0.190274,0.0818069,0.435987,0.0425558,0.0512971
498,30-5,0.115385,0.176471,0,0,0.0769231,0.114286,1,0.4,0.0909091,...,0.0054713,0.0767209,0.197086,0.0830699,0,0.118322,0.0318877,0.227632,0.0258547,0.0353384


In [73]:
start_time = time.time()

#Step2 Split workload_C by creating different dataframes

workload_C_ids = norm_workload_C["workload id"].unique()
workload_C_list = []
for num,workload_C_id in enumerate(workload_C_ids,start=1):
    tmp_workload = norm_workload_C[norm_workload_C['workload id'].isin([workload_C_id])]
    workload_C_list.append(tmp_workload)
    exec("workload_C_%s = tmp_workload"%num)

print("--- %s seconds ---" % (time.time() - start_time))

--- 0.048868656158447266 seconds ---


In [74]:
start_time = time.time()

#Step3 Load workload_C target

workload_C_target=pd.read_csv('../../test.csv')

tmp_workload_C_knobs_metrics_t = (workload_C_target[knobs_list]).values
tmp_fill_value = np.zeros((100,313-12))

tmp_workload_C_knobs_metrics_t_withfill = np.hstack((tmp_workload_C_knobs_metrics_t,tmp_fill_value))

Cworkload_knobs_metrics_Norm_t = dataScalar.transform(tmp_workload_C_knobs_metrics_t_withfill)
Ctmp_workload_id_t = (workload_C_target[work_id_name]).values

Cworkload_knobs_metrics_Norm_id_t = np.hstack((Ctmp_workload_id_t,Cworkload_knobs_metrics_Norm_t))
norm_workload_C_t = pd.DataFrame(data=Cworkload_knobs_metrics_Norm_id_t,columns=col_list)

workload_C_ids_t = norm_workload_C_t["workload id"].unique()
workload_C_list_t = []
for num,workload_C_id in enumerate(workload_C_ids_t,start=1):
    tmp_workload = norm_workload_C_t[norm_workload_C_t['workload id'].isin([workload_C_id])]
    workload_C_list_t.append(tmp_workload)
    exec("workload_C_%s = tmp_workload"%num)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.06382894515991211 seconds ---


In [75]:
#Step4 FA for all workloads together Function
from sklearn.decomposition import FactorAnalysis
from sklearn.preprocessing import normalize


start_time = time.time()


def FAforAllworkloads(n_c,frame):
    all_metrics_data = frame.values

    all_metrics_data_Trans = all_metrics_data.T
    tmp_all_transformer = FactorAnalysis(n_components=n_c, random_state=0)
    tmp_workload_A_transformed = tmp_all_transformer.fit_transform(all_metrics_data_Trans)
    
    return tmp_workload_A_transformed
    

In [76]:
#Step5 K-means for all workloads together
from sklearn.cluster import KMeans

def KmeanForallworkloads(n_k,faworkloads):
    workload_A_list_array_Kmeans = KMeans(n_clusters=n_k, random_state=0).fit(faworkloads)
    cluster_centers = workload_A_list_array_Kmeans.cluster_centers_
    tmp_centers_index = []
    tmp_centers_metrics_name = []
    
    for i in range(n_k):
        tmp_center = cluster_centers[i]
        tmp_center_diff = tmp_center - faworkloads
        tmp_center_distance = np.sum((tmp_center_diff * tmp_center_diff),axis=1)
        tmp_center_min_idx = np.argmin(tmp_center_distance)
        tmp_centers_index.append(tmp_center_min_idx)
    
    for j in range(n_k):
        tmp_mname = metrics_name[tmp_centers_index[j]]
        #print(tmp_mname)
        tmp_centers_metrics_name.append(tmp_mname)
        #print(tmp_centers_index)
        #print(tmp_centers_metrics_name)

    tmp_centers_metrics_name = ['latency'] + tmp_centers_metrics_name
    print(tmp_centers_metrics_name)
    num_M = 1 + n_k
    workload_list_centers_metrics_name = []
    for n in range(58):
        workload_list_centers_metrics_name.append(tmp_centers_metrics_name)
        
    return num_M,tmp_centers_metrics_name

In [77]:
#Step7 Building GPR (apply for all workloads)
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
from sklearn.gaussian_process.kernels import RBF
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import MinMaxScaler



def BuildGPRForall(kernel_t,workload_A,centers_metrics_name):
    tmp_kernel = kernel_t
    
    workload_A_GPRmodels_lists = []
    workload_A_scalar_lists = []
    workload_A_normalized_m = []
    
    for i in range(58):
        tmp_GPRmodels_lists = []
        tmp_scalar_lists = []
        tmp_normalized_m = []
    
    
        tmp_Pruning_workload_A_X = ((workload_A[i])[knobs_list]).values
    
        tmp_num_centers = len(centers_metrics_name)
        
        for j in range(tmp_num_centers):
        
            tmp_Pruning_workload_A_y_m = ((workload_A[i])[centers_metrics_name[j]]).values
            tmp_workload_A_gpr_m = GaussianProcessRegressor(kernel=tmp_kernel).fit(tmp_Pruning_workload_A_X, tmp_Pruning_workload_A_y_m)
            tmp_GPRmodels_lists.append(tmp_workload_A_gpr_m)
        
        workload_A_GPRmodels_lists.append(tmp_GPRmodels_lists)
    return workload_A_GPRmodels_lists
#print(workload_A_normalized_m)

In [78]:
#Step4 workload mapping Test for all workload in workload B

def FindMostSimliarIndexForTarget(workload_A,workload_mapping,GPRmodelList,center_metric_name,t_kernel,n_m):
    all_distance_list = []
    for i in range(100):
        tmp_test_workload_B_mapping = workload_mapping[i]
        tmp_test_workload_B_mapping_cfigs = ((tmp_test_workload_B_mapping)[knobs_list]).values
    
        tmp_distance_list = []
    
        for j in range(58):
            tmp_test_workload_A = workload_A[j]
            tmp_test_workload_A_gpr_model = GPRmodelList[j]   
            tmp_test_workload_A_cfigs = ((tmp_test_workload_A)[knobs_list]).values
        
            tmp_corr_metric_values = []
        
            for k in range(5):
                tmp_data_point = tmp_test_workload_B_mapping_cfigs[k]
                tmp_check = np.where((tmp_data_point==tmp_test_workload_A_cfigs[:,None]).all(-1))
            
                if len(tmp_check[0]) == 0:
                    tmp_gpr_predic = []
                    for v in range(nk):
                        tmp_gpr_predic.append(tmp_test_workload_A_gpr_model[v].predict([tmp_data_point])[0])
                    tmp_corr_metric_values.append(tmp_gpr_predic)
            
                else:
                    tmp_gpr_predic = []
                    tmp_row_idx = tmp_check[0][0]
                    for n in range(n_m):
                        tmp_gpr_predic.append((((tmp_test_workload_A)[[center_metric_name[n]]]).values)[tmp_row_idx])
                    tmp_corr_metric_values.append(tmp_gpr_predic)
        
            tmp_test_workload_B_mapping_metrics_norm_value_list = []
            for m in range(nk):
                tmp_test_workload_B_mapping_metrics_value = tmp_test_workload_B_mapping[[center_metric_name[m]]].values

                tmp_test_workload_B_mapping_metrics_norm_value_list.append(tmp_test_workload_B_mapping_metrics_value)


            tmp_test_workload_B_mapping_metrics_norm_value_arry = np.asarray(tmp_test_workload_B_mapping_metrics_norm_value_list).reshape((n_m,5))       
            tmp_corr_metric_values_arry = np.asarray(tmp_corr_metric_values).reshape((5,n_m)).T

            up_min_list = np.min(tmp_test_workload_B_mapping_metrics_norm_value_arry,axis=0)
            down_min_list = np.min(tmp_corr_metric_values_arry,axis=0)
            tot_min_list = np.min(np.vstack((up_min_list,down_min_list)),axis=0)
        
            up_max_list = np.max(tmp_test_workload_B_mapping_metrics_norm_value_arry,axis=0)
            down_max_list = np.max(tmp_corr_metric_values_arry,axis=0)
            tot_max_list = np.max(np.vstack((up_max_list,down_max_list)),axis=0)

            tmp_test_workload_B_mapping_metrics_norm_value_arry_norm = np.divide((tmp_test_workload_B_mapping_metrics_norm_value_arry - tot_min_list),(tot_max_list - tot_min_list))
            tmp_corr_metric_values_arry_norm = np.divide((tmp_corr_metric_values_arry - tot_min_list),(tot_max_list - tot_min_list))

            tmp_distance = np.sum((tmp_test_workload_B_mapping_metrics_norm_value_arry_norm-tmp_corr_metric_values_arry_norm)*(tmp_test_workload_B_mapping_metrics_norm_value_arry_norm-tmp_corr_metric_values_arry_norm))
            tmp_distance_list.append(tmp_distance)
     
        all_distance_list.append(tmp_distance_list)


    
    all_min_idx_list = []
    for i in range(100):
        tmp_idx = all_distance_list[i].index(min(all_distance_list[i]))
        all_min_idx_list.append(tmp_idx)
    return all_min_idx_list

In [79]:
#Step5 Concatenate trances from the observed target workload and the nearest neighbor workload for all workload B

def Concatenate(workload_A,workload_mapping,center_metric_name,t_kernel,min_idx_list):
    predictGPRmodel_list = []
    #predictScalar_list = []
    #inputScalar_list = []


    for i in range(100):
        
        #print(i)
        tmp_similar_idx = min_idx_list[i]
        tmp_similar_workload = workload_A[tmp_similar_idx]
        tmp_similar_workload_configs = ((tmp_similar_workload)[knobs_list]).values
        tmp_similar_workload_metrics = ((tmp_similar_workload)[[center_metric_name[0]]]).values
    
        tmp_Augmented_pruned_target_configs = np.array(tmp_similar_workload_configs)
        tmp_Augmented_pruned_target_metrics = np.array(tmp_similar_workload_metrics)
    
        tmp_workload_B_configs = workload_mapping[i][knobs_list].values
        tmp_workload_B_metrics = workload_mapping[i][[center_metric_name[0]]].values
    
        for j in range(5):
            tmp_test_workload_B_mapping_cfigs = tmp_workload_B_configs[j]
            tmp_test_workload_B_mapping_metrics_with_similar = tmp_workload_B_metrics[j]
            tmp_check = np.where((tmp_test_workload_B_mapping_cfigs==tmp_similar_workload_configs[:,None]).all(-1))
        
            if len(tmp_check[0]) == 0:
            
                tmp_Augmented_pruned_target_configs = np.append(tmp_Augmented_pruned_target_configs,[tmp_test_workload_B_mapping_cfigs],axis=0)
                tmp_Augmented_pruned_target_metrics = np.append(tmp_Augmented_pruned_target_metrics,[tmp_test_workload_B_mapping_metrics_with_similar],axis=0)
            else:
        
                tmp_row_idx = tmp_check[0][0]
                tmp_Augmented_pruned_target_metrics[tmp_row_idx] = tmp_test_workload_B_mapping_metrics_with_similar
    
        test_workload_B_gpr_o = GaussianProcessRegressor(kernel=t_kernel).fit(tmp_Augmented_pruned_target_configs, tmp_Augmented_pruned_target_metrics)
        predictGPRmodel_list.append(test_workload_B_gpr_o)
        
    return predictGPRmodel_list

In [144]:
#Predict
def PredictMAPE(workload_target,predGPRmodelList,dscalar,center_metric_name,min_idx_list):
    rightSet = {}

    totPercent = 0
    right_count = 0
    for i in range(100):
        tmp_target_config = workload_target[i][knobs_list].values
        tmp_max = dscalar.data_max_[12]
        tmp_min = dscalar.data_min_[12]
        tmp_predict = predGPRmodelList[i].predict(tmp_target_config)
        
        #tmp_predict_re = tmp_predict * (tmp_max - tmp_min) + tmp_min
        #tmp_ground_Truth = workload_B_target[i][[center_metric_name[0]]].values * (tmp_max - tmp_min) + tmp_min
        tmp_predict_re = latencyScalar.inverse_transform(tmp_predict)
        tmp_ground_Truth = latencyScalar.inverse_transform(workload_target[i][[center_metric_name[0]]].values)
        
        tmp_diff = np.abs(tmp_predict_re - tmp_ground_Truth)
        #print(tmp_predict_re)
        #print(tmp_ground_Truth)
        totPercent += tmp_diff/tmp_ground_Truth
        if (tmp_diff/tmp_ground_Truth) < 0.3:
            right_count += 1
            rightSet[i] = min_idx_list[i]
    #print('------------')

    print('MAPE: ',totPercent/100)
    print('Right Predict with absolute percentage error less than 0.3: ', right_count)
    return totPercent,rightSet

In [81]:
#Predict
def PredictMAPEwithoutGroundTruth(workload_target,predGPRmodelList,dscalar,center_metric_name,min_idx_list):
    predictList = []
    for i in range(100):
        tmp_target_config = workload_target[i][knobs_list].values
        tmp_max = dscalar.data_max_[12]
        tmp_min = dscalar.data_min_[12]
        tmp_predict = predGPRmodelList[i].predict(tmp_target_config)
        tmp_predict_re = latencyScalar.inverse_transform(tmp_predict)
        #tmp_predict_re = tmp_predict * (tmp_max - tmp_min) + tmp_min
        predictList.append(tmp_predict_re)
        
    return predictList

In [163]:
kernel_1 = 4.0 * RBF(length_scale=1.8) + WhiteKernel()
a1 = FAforAllworkloads(170,tmpframe)
nk,b1 = KmeanForallworkloads(2,a1)
d1 = BuildGPRForall(kernel_1,workload_A_list,b1)
e1 = FindMostSimliarIndexForTarget(workload_A_list,workload_B_mapping_data_list[0],d1,b1,kernel_1,nk)
f1 = Concatenate(workload_A_list,workload_B_mapping_data_list[0],b1,kernel_1,e1)
h1 = PredictMAPE(workload_B_target_list[0],f1,dataScalar,b1,e1)

['latency', 'driver.LiveListenerBus.queue.eventLog.size.avg', 'worker_1.JFS_Filespace_%Used./home']
predict:  [[13.11781192]]
True:  [[7.923]]
predict:  [[248.7926266]]
True:  [[260.62625]]
predict:  [[100.5832102]]
True:  [[109.0855]]
predict:  [[70.88051155]]
True:  [[61.997]]
predict:  [[85.93540538]]
True:  [[89.78475]]
predict:  [[16.91810771]]
True:  [[24.4785]]
predict:  [[10.64251546]]
True:  [[4.507125]]
predict:  [[130.18767545]]
True:  [[107.993]]
predict:  [[28.34440792]]
True:  [[42.31425]]
predict:  [[30.8878284]]
True:  [[52.674]]
predict:  [[6.85670627]]
True:  [[3.084]]
predict:  [[11.04683576]]
True:  [[9.70575]]
predict:  [[41.71314919]]
True:  [[46.812]]
predict:  [[6.70531553]]
True:  [[24.076]]
predict:  [[6.89108395]]
True:  [[8.29225]]
predict:  [[0.49535665]]
True:  [[18.494]]
predict:  [[40.16746086]]
True:  [[59.1705]]
predict:  [[218.63772767]]
True:  [[241.722]]
predict:  [[50.37330976]]
True:  [[39.0855]]
predict:  [[22.1276254]]
True:  [[13.14875]]
predic

In [158]:
workload_B_target_list_pair = []
for i in range(100):
    workload_B_target_list_pair.append(workload_A_ids[e1[i]])
    
workload_B_target_list_array = np.array(workload_B_target_list_pair)
workload_B_nearset_pair = np.vstack((workload_B_ids,workload_B_target_list_array))
workload_B_nearset_pair_re = workload_B_nearset_pair.T
workload_B_nearset_pair_re_pd = pd.DataFrame(data=workload_B_nearset_pair_re,columns=['workload_B workload id','offline workload id'])

In [161]:
workload_B_nearset_pair_re

array([['1-1', '1-7'],
       ['2-3', '30-0'],
       ['3-3', '3-2'],
       ['4-8', '3-2'],
       ['5-5', '5-1'],
       ['6-8', '6-4'],
       ['7-0', '7-3'],
       ['8-7', '8-6'],
       ['9-7', '20-4'],
       ['10-1', '10-0'],
       ['11-0', '1-7'],
       ['12-5', '12-8'],
       ['13-7', '16-3'],
       ['14-2', '14-0'],
       ['15-1', '15-3'],
       ['16-2', '16-4'],
       ['17-0', '17-5'],
       ['18-7', '18-8'],
       ['19-6', '19-4'],
       ['20-7', '26-5'],
       ['21-3', '21-7'],
       ['22-8', '22-7'],
       ['23-5', '23-0'],
       ['24-8', '24-5'],
       ['25-4', '25-7'],
       ['26-2', '25-1'],
       ['27-3', '19-4'],
       ['28-2', '22-2'],
       ['29-2', '7-3'],
       ['30-7', '3-0'],
       ['1-3', '1-7'],
       ['2-8', '2-0'],
       ['3-8', '3-2'],
       ['4-2', '30-0'],
       ['5-0', '5-6'],
       ['6-3', '6-4'],
       ['7-8', '11-2'],
       ['8-0', '8-6'],
       ['9-2', '29-1'],
       ['10-2', '10-0'],
       ['11-1', '1-7'],
       ['1

In [82]:
kernel_1 = 1.0 * RBF(length_scale=1.0) + WhiteKernel()
num_FA_reduce = [170]
num_K = [2]

result_set = {}
metric_set = {}

for i in range(len(num_FA_reduce)):
    for j in range(len(num_K)):
        a = FAforAllworkloads(num_FA_reduce[i],tmpframe)
        nk,b = KmeanForallworkloads(num_K[j],a)
        
        metric_set[(i,j)] = b
        
        d = BuildGPRForall(kernel_1,workload_A_list,b)
        
        tmp_MAPE = 0
        for n in range(6):
            e = FindMostSimliarIndexForTarget(workload_A_list,workload_B_mapping_data_list[n],d,b,kernel_1,nk)
            f = Concatenate(workload_A_list,workload_B_mapping_data_list[n],b,kernel_1,e)
            g,h = PredictMAPE(workload_B_target_list[n],f,dataScalar,b,e)
            tmp_MAPE += g[0][0]
        result_set[(i,j)] = tmp_MAPE/6
            #print(g)

['latency', 'driver.LiveListenerBus.queue.eventLog.size.avg', 'worker_1.JFS_Filespace_%Used./home']
MAPE:  [[0.35849892]]
Right Predict with absolute percentage error less than 0.3:  54
MAPE:  [[0.32891837]]
Right Predict with absolute percentage error less than 0.3:  61
MAPE:  [[0.32159553]]
Right Predict with absolute percentage error less than 0.3:  66
MAPE:  [[0.389459]]
Right Predict with absolute percentage error less than 0.3:  59
MAPE:  [[0.38004371]]
Right Predict with absolute percentage error less than 0.3:  67
MAPE:  [[0.39959436]]
Right Predict with absolute percentage error less than 0.3:  60


In [83]:

#Test File
#workload_C_list
#workload_C_list_t


In [84]:
kernel_1 = 1.0 * RBF(length_scale=1.0) + WhiteKernel()
a = FAforAllworkloads(170,tmpframe)
nk,b = KmeanForallworkloads(2,a)
d = BuildGPRForall(kernel_1,workload_A_list,b)
e = FindMostSimliarIndexForTarget(workload_A_list,workload_C_list,d,b,kernel_1,nk)
f = Concatenate(workload_A_list,workload_C_list,b,kernel_1,e)
h = PredictMAPEwithoutGroundTruth(workload_C_list_t,f,dataScalar,b,e)

['latency', 'driver.LiveListenerBus.queue.eventLog.size.avg', 'worker_1.JFS_Filespace_%Used./home']


In [119]:
workload_c_target_list = []
for i in range(100):
    workload_c_target_list.append(workload_A_ids[e[i]])
    
workload_c_target_list_array = np.array(workload_c_target_list)
workload_c_nearset_pair = np.vstack((workload_C_ids,workload_c_target_list_array))
workload_c_nearset_pair_re = workload_c_nearset_pair.T
workload_c_nearset_pair_re_pd = pd.DataFrame(data=workload_c_nearset_pair_re,columns=['workload_C workload id','offline workload id'])

In [121]:
#e
#h

In [138]:
predcit_list_col = ['workload id','k1','k2','k3','k4','k5','k6','k7','k8','s1','s2','s3','s4','latency prediction']
workloadC_predict_array = np.array(h).reshape(100,1)
workload_C_withpredict = np.hstack((tmp_workload_C_knobs_metrics_t,workloadC_predict_array))
workload_C_withpredict_id = np.hstack((Ctmp_workload_id_t,workload_C_withpredict))
pd_workload_C_withpredict_id = pd.DataFrame(data=workload_C_withpredict_id,columns=predcit_list_col)

In [164]:
h

[array([[7.41749497]]),
 array([[49.6054342]]),
 array([[12.78416102]]),
 array([[46.64942341]]),
 array([[38.36996825]]),
 array([[12.03853961]]),
 array([[26.60731563]]),
 array([[12.38288888]]),
 array([[27.76541388]]),
 array([[6.68359839]]),
 array([[17.69326677]]),
 array([[2.49560164]]),
 array([[24.11256447]]),
 array([[66.41759219]]),
 array([[7.73205129]]),
 array([[376.70136547]]),
 array([[6.88804997]]),
 array([[908.06369355]]),
 array([[167.5640096]]),
 array([[73.17439983]]),
 array([[52.34041295]]),
 array([[134.58207543]]),
 array([[8.53764057]]),
 array([[25.93531432]]),
 array([[11.42922734]]),
 array([[24.41640409]]),
 array([[39.40531775]]),
 array([[6.88316398]]),
 array([[6.88067423]]),
 array([[46.40569918]]),
 array([[13.20276367]]),
 array([[46.69249807]]),
 array([[31.75410713]]),
 array([[5.97258916]]),
 array([[17.80251321]]),
 array([[24.25463394]]),
 array([[94.89286812]]),
 array([[8.17254316]]),
 array([[12.31015727]]),
 array([[41.81541186]]),
 array([